In [1]:
import os
import random
import gym
import pylab
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K    


import GameHard
import DisplayIA

env = GameHard.Game()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def OurModel(input_shape, action_space, dueling):
    print(action_space)
    """
    frames_input = keras.layers.Input(input_shape, name='frames')
    normalized = keras.layers.Lambda(lambda x: x / 1000.0)(frames_input)
    X = normalized

    # 'Dense' is the basic form of a neural network layer
    # Input Layer of state size(4) and Hidden Layer with 512 nodes
    X = Dense(300, input_shape=input_shape, activation="relu", kernel_initializer=keras.initializers.VarianceScaling())(X)

    # Hidden layer with 256 nodes
    X = Dense(200, activation="relu", kernel_initializer=keras.initializers.VarianceScaling())(X)
    
    # Hidden layer with 64 nodes
    X = Dense(300, activation="relu", kernel_initializer=keras.initializers.VarianceScaling())(X)

    if dueling:
        state_value = Dense(1, kernel_initializer='he_uniform')(X)
        state_value = Lambda(lambda s: K.expand_dims(s[:, 0], -1), output_shape=(action_space,))(state_value)

        action_advantage = Dense(action_space, kernel_initializer='he_uniform')(X)
        action_advantage = Lambda(lambda a: a[:, :] - K.mean(a[:, :], keepdims=True), output_shape=(action_space,))(action_advantage)

        X = Add()([state_value, action_advantage])
    else:
        # Output Layer with # of actions: 2 nodes (left, right)
        X = Dense(action_space, activation="linear", kernel_initializer='he_uniform')(X)
"""
    frames_input = keras.layers.Input(input_shape, name='frames')

    # Assuming that the input frames are still encoded from 0 to 255. Transforming to [0, 1].
    normalized = keras.layers.Lambda(lambda x: x / 1000.0)(frames_input)

    flatten = keras.layers.Flatten()(normalized)
    dense_1 = keras.layers.Dense(300, activation="relu", kernel_initializer=keras.initializers.VarianceScaling())(flatten)
    dense_2 = keras.layers.Dense(200, activation="relu", kernel_initializer=keras.initializers.VarianceScaling())(dense_1)
    dense_3 = keras.layers.Dense(300, activation="relu", kernel_initializer=keras.initializers.VarianceScaling())(dense_2)
    # "The output layer is a fully-connected linear layer with a single output for each valid action."
    output = keras.layers.Dense(3)(dense_3)
        
        
    model = Model(inputs = frames_input, outputs = output, name='YoushiModel')
    model.compile(loss=keras.losses.Huber(reduction="none"), optimizer=keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0, epsilon=0.00001, centered=True), metrics=["accuracy"])

    model.summary()
    return model


In [3]:
#PER memory
class SumTree(object):
    data_pointer = 0
    
    # Here we initialize the tree with all nodes = 0, and initialize the data with all values = 0
    def __init__(self, capacity):
        # Number of leaf nodes (final nodes) that contains experiences
        self.capacity = capacity
        
        # Generate the tree with all nodes values = 0
        # To understand this calculation (2 * capacity - 1) look at the schema below
        # Remember we are in a binary node (each node has max 2 children) so 2x size of leaf (capacity) - 1 (root node)
        # Parent nodes = capacity - 1
        # Leaf nodes = capacity
        self.tree = np.zeros(2 * capacity - 1)
        
        # Contains the experiences (so the size of data is capacity)
        self.data = np.zeros(capacity, dtype=object)
        
        
    def add(self, priority, data):
        # Look at what index we want to put the experience
        tree_index = self.data_pointer + self.capacity - 1

        """ tree:
                    0
                   / \
                  0   0
                 / \ / \
        tree_index  0 0  0  We fill the leaves from left to right
        """

        # Update data frame
        self.data[self.data_pointer] = data

        # Update the leaf
        self.update (tree_index, priority)

        # Add 1 to data_pointer
        self.data_pointer += 1

        if self.data_pointer >= self.capacity:  # If we're above the capacity, we go back to first index (we overwrite)
            self.data_pointer = 0


    def update(self, tree_index, priority):
        # Change = new priority score - former priority score
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority

        # then propagate the change through tree
        # this method is faster than the recursive loop
        while tree_index != 0:
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
            
            
            
    def get_leaf(self, v):
        parent_index = 0

        while True:
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1

            # If we reach bottom, end the search
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            else: # downward search, always search for a higher priority node
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index

        data_index = leaf_index - self.capacity + 1

        return leaf_index, self.tree[leaf_index], self.data[data_index]

    @property
    def total_priority(self):
        return self.tree[0] # Returns the root node 
    
    
class Memory(object):  # stored as ( state, action, reward, next_state ) in SumTree
    PER_e = 0.01  # Hyperparameter that we use to avoid some experiences to have 0 probability of being taken
    PER_a = 0.6  # Hyperparameter that we use to make a tradeoff between taking only exp with high priority and sampling randomly
    PER_b = 0.4  # importance-sampling, from initial value increasing to 1
    
    PER_b_increment_per_sampling = 0.001
    
    absolute_error_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        # Making the tree 
        self.tree = SumTree(capacity)
        
    def store(self, experience):
        # Find the max priority
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])

        # If the max priority = 0 we can't put priority = 0 since this experience will never have a chance to be selected
        # So we use a minimum priority
        if max_priority == 0:
            max_priority = self.absolute_error_upper

        self.tree.add(max_priority, experience)   # set the max priority for new priority
    
    
    def sample(self, n):
        # Create a minibatch array that will contains the minibatch
        minibatch = []

        b_idx = np.empty((n,), dtype=np.int32)

        # Calculate the priority segment
        # Here, as explained in the paper, we divide the Range[0, ptotal] into n ranges
        priority_segment = self.tree.total_priority / n       # priority segment

        for i in range(n):
            # A value is uniformly sample from each range
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)

            # Experience that correspond to each value is retrieved
            index, priority, data = self.tree.get_leaf(value)

            b_idx[i]= index

            minibatch.append([data[0],data[1],data[2],data[3],data[4]])

        return b_idx, minibatch
    
    
    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PER_e  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.absolute_error_upper)
        ps = np.power(clipped_errors, self.PER_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [4]:
class DQNAgent:
    def __init__(self, env_name):
        self.env_name = env_name       
        self.env = GameHard.Game()
        # by default, CartPole-v1 has max episode steps = 500
        #self.env._max_episode_steps = 40000
        self.state_size = [15, 2]
        self.action_size = 3

        self.EPISODES = 30000
        memory_size = 100000
        self.MEMORY = Memory(memory_size)
        self.memory = deque(maxlen=2000)
        self.gamma = 0.99    # discount rate

        # EXPLORATION HYPERPARAMETERS for epsilon and epsilon greedy strategy
        self.epsilon = 1.0 # exploration probability at start
        self.epsilon_min = 0.01 # minimum exploration probability
        self.epsilon_decay = 0.00005 # exponential decay rate for exploration prob
        
        self.batch_size = 64

        # defining model parameters
        self.ddqn = False # use doudle deep q network
        self.Soft_Update = False # use soft parameter update
        self.dueling = True # use dealing netowrk
        self.epsilon_greedy = False # use epsilon greedy strategy
        self.USE_PER = True

        self.TAU = 0.1 # target network soft update hyperparameter

        self.Save_Path = 'Models'
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.scores, self.episodes, self.average = [], [], []

        self.Model_name = os.path.join(self.Save_Path, self.env_name+"_e_greedy.h5")
        
        # create main model and target model
        self.model = OurModel(input_shape=self.state_size, action_space = self.action_size, dueling = self.dueling)
        self.target_model = OurModel(input_shape=self.state_size, action_space = self.action_size, dueling = self.dueling)

    # after some time interval update the target model to be same with model
    def update_target_model(self):
        if not self.Soft_Update and self.ddqn:
            self.target_model.set_weights(self.model.get_weights())
            return
        if self.Soft_Update and self.ddqn:
            q_model_theta = self.model.get_weights()
            target_model_theta = self.target_model.get_weights()
            counter = 0
            for q_weight, target_weight in zip(q_model_theta, target_model_theta):
                target_weight = target_weight * (1-self.TAU) + q_weight * self.TAU
                target_model_theta[counter] = target_weight
                counter += 1
            self.target_model.set_weights(target_model_theta)

    def remember(self, state, action, reward, next_state, done):
        experience = state, action, reward, next_state, done
        if self.USE_PER:
            self.MEMORY.store(experience)
        else:
            self.memory.append((experience))

    def act(self, state, decay_step):
        # EPSILON GREEDY STRATEGY
        if self.epsilon_greedy:
        # Here we'll use an improved version of our epsilon greedy strategy for Q-learning
            explore_probability = self.epsilon_min + (self.epsilon - self.epsilon_min) * np.exp(-self.epsilon_decay * decay_step)
        # OLD EPSILON STRATEGY
        else:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= (1-self.epsilon_decay)
            explore_probability = self.epsilon
    
        if explore_probability > np.random.rand():
            # Make a random action (exploration)
            action = random.randrange(self.action_size)
            return action, explore_probability
        else:
            # Get action from Q-network (exploitation)
            # Estimate the Qs values state
            # Take the biggest Q value (= the best action)
            action = np.argmax(self.model.predict(state[np.newaxis]))
            return action, explore_probability

    def replay(self):
        if self.USE_PER:
            tree_idx, minibatch = self.MEMORY.sample(self.batch_size)
        else:
            minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))

        state = np.zeros((self.batch_size, 15, 2))
        next_state = np.zeros((self.batch_size, 15, 2))
        action, reward, done = [], [], []

        # do this before prediction
        # for speedup, this could be done on the tensor level
        # but easier to understand using a loop
        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        # do batch prediction to save speed
        # predict Q-values for starting state using the main network
        target = self.model.predict(state)
        target_old = np.array(target)
        # predict best action in ending state using the main network
        target_next = self.model.predict(next_state)
        # predict Q-values for ending state using the target network
        target_val = self.target_model.predict(next_state)

        for i in range(len(minibatch)):
            # correction on the Q value for the action used
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                if self.ddqn: # Double - DQN
                    # current Q Network selects the action
                    # a'_max = argmax_a' Q(s', a')
                    a = np.argmax(target_next[i])
                    # target Q Network evaluates the action
                    # Q_max = Q_target(s', a'_max)
                    target[i][action[i]] = reward[i] + self.gamma * (target_val[i][a])   
                else: # Standard - DQN
                    # DQN chooses the max Q value among next actions
                    # selection and evaluation of action is on the target Q Network
                    # Q_max = max_a' Q_target(s', a')
                    target[i][action[i]] = reward[i] + self.gamma * (np.amax(target_next[i]))

        if self.USE_PER:
            indices = np.arange(self.batch_size, dtype=np.int32)
            #absolute_errors = np.mean(np.abs(target_old[indices, np.array(action)]-target[indices, np.array(action)]), axis = 1)
            absolute_errors = np.abs(target_old[i]-target[i])
            # Update priority
            self.MEMORY.batch_update(tree_idx, absolute_errors)

        # Train the Neural Network with batches
        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)

    def load(self, name):
        self.model = load_model(name)

    def save(self, name):
        self.model.save(name)

    pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        pylab.plot(self.episodes, self.average, 'r')
        pylab.plot(self.episodes, self.scores, 'b')
        pylab.ylabel('Score', fontsize=18)
        pylab.xlabel('Steps', fontsize=18)
        dqn = 'DQN_'
        softupdate = ''
        dueling = ''
        greedy = ''
        PER = ''
        if self.ddqn: dqn = 'DDQN_'
        if self.Soft_Update: softupdate = '_soft'
        if self.dueling: dueling = '_Dueling'
        if self.epsilon_greedy: greedy = '_Greedy'
        if self.USE_PER: PER = '_PER'
        try:
            pylab.savefig(dqn+self.env_name+softupdate+dueling+greedy+PER+".png")
        except OSError:
            pass

        return str(self.average[-1])[:5]
    
    def run(self):
        decay_step = 0
        for e in range(self.EPISODES):
            state = self.env.reset()
            #state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                #self.env.render()
                decay_step += 1
                action, explore_probability = self.act(state, decay_step)
                next_state, reward, done, _ = self.env.step(action)
                #next_state = np.reshape(next_state, [1, self.state_size])
                if not done or i == 10000-1:
                    reward = reward
                else:
                    reward = -100
                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1
                if done:
                    # every step update target model
                    self.update_target_model()
                    
                    # every episode, plot the result
                    average = self.PlotModel(i, e)
                     
                    print("episode: {}/{}, score: {}, e: {:.2}, average: {}".format(e, self.EPISODES, i, explore_probability, average))
                    if e % 1000 == 0:
                        print("Saving trained model to", self.Model_name)
                        self.save(self.Model_name)
                        break
                self.replay()
        #self.env.close()

    def test(self):
        self.load(self.Model_name)
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                #self.env.render()
                action = np.argmax(self.model.predict(state))
                next_state, reward, done, _ = self.env.step(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += 1
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, i))
                    break

<Figure size 1296x648 with 0 Axes>

In [5]:
if __name__ == "__main__":
    env_name = 'CartPole-v1'
    agent = DQNAgent(env_name)
    agent.run()

3
Model: "YoushiModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
frames (InputLayer)          [(None, 15, 2)]           0         
_________________________________________________________________
lambda (Lambda)              (None, 15, 2)             0         
_________________________________________________________________
flatten (Flatten)            (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 300)               9300      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               60300     
_________________________________________________________________
dense_3 (Dense)              (None, 3)               

episode: 104/30000, score: 123, e: 0.58, average: 101.2
episode: 105/30000, score: 91, e: 0.58, average: 101.4
episode: 106/30000, score: 115, e: 0.58, average: 101.4
episode: 107/30000, score: 108, e: 0.58, average: 100.0
episode: 108/30000, score: 91, e: 0.57, average: 100.1
episode: 109/30000, score: 144, e: 0.57, average: 101.4
episode: 110/30000, score: 123, e: 0.57, average: 102.2
episode: 111/30000, score: 66, e: 0.56, average: 101.6
episode: 112/30000, score: 66, e: 0.56, average: 100.6
episode: 113/30000, score: 83, e: 0.56, average: 100.9
episode: 114/30000, score: 66, e: 0.56, average: 99.98
episode: 115/30000, score: 251, e: 0.55, average: 103.3
episode: 116/30000, score: 173, e: 0.55, average: 104.9
episode: 117/30000, score: 83, e: 0.54, average: 104.6
episode: 118/30000, score: 189, e: 0.54, average: 106.7
episode: 119/30000, score: 83, e: 0.54, average: 107.0
episode: 120/30000, score: 83, e: 0.53, average: 106.8
episode: 121/30000, score: 83, e: 0.53, average: 107.0
ep

episode: 253/30000, score: 67, e: 0.25, average: 122.3
episode: 254/30000, score: 115, e: 0.25, average: 121.2
episode: 255/30000, score: 77, e: 0.25, average: 119.8
episode: 256/30000, score: 99, e: 0.25, average: 120.4
episode: 257/30000, score: 137, e: 0.25, average: 121.7
episode: 258/30000, score: 75, e: 0.25, average: 121.7
episode: 259/30000, score: 99, e: 0.25, average: 122.0
episode: 260/30000, score: 83, e: 0.25, average: 121.3
episode: 261/30000, score: 91, e: 0.24, average: 118.8
episode: 262/30000, score: 130, e: 0.24, average: 118.4
episode: 263/30000, score: 66, e: 0.24, average: 117.7
episode: 264/30000, score: 251, e: 0.24, average: 121.2
episode: 265/30000, score: 107, e: 0.24, average: 121.5
episode: 266/30000, score: 66, e: 0.24, average: 117.5
episode: 267/30000, score: 107, e: 0.24, average: 118.3
episode: 268/30000, score: 159, e: 0.23, average: 120.1
episode: 269/30000, score: 276, e: 0.23, average: 124.3
episode: 270/30000, score: 99, e: 0.23, average: 120.2
ep

episode: 400/30000, score: 259, e: 0.068, average: 197.6
episode: 401/30000, score: 116, e: 0.068, average: 198.4
episode: 402/30000, score: 367, e: 0.067, average: 200.4
episode: 403/30000, score: 137, e: 0.066, average: 200.6
episode: 404/30000, score: 142, e: 0.066, average: 201.3
episode: 405/30000, score: 99, e: 0.066, average: 198.4
episode: 406/30000, score: 273, e: 0.065, average: 201.3
episode: 407/30000, score: 182, e: 0.064, average: 202.8
episode: 408/30000, score: 128, e: 0.064, average: 200.9
episode: 409/30000, score: 91, e: 0.063, average: 200.3
episode: 410/30000, score: 303, e: 0.062, average: 197.9
episode: 411/30000, score: 212, e: 0.062, average: 197.5
episode: 412/30000, score: 66, e: 0.062, average: 195.6
episode: 413/30000, score: 228, e: 0.061, average: 198.5
episode: 414/30000, score: 115, e: 0.06, average: 198.7
episode: 415/30000, score: 296, e: 0.06, average: 199.9
episode: 416/30000, score: 123, e: 0.059, average: 198.2
episode: 417/30000, score: 273, e: 0

episode: 545/30000, score: 123, e: 0.019, average: 179.4
episode: 546/30000, score: 84, e: 0.019, average: 179.8
episode: 547/30000, score: 66, e: 0.019, average: 179.8
episode: 548/30000, score: 156, e: 0.019, average: 179.7
episode: 549/30000, score: 107, e: 0.019, average: 178.7
episode: 550/30000, score: 66, e: 0.019, average: 173.9
episode: 551/30000, score: 115, e: 0.019, average: 174.5
episode: 552/30000, score: 115, e: 0.018, average: 175.3
episode: 553/30000, score: 83, e: 0.018, average: 175.7
episode: 554/30000, score: 228, e: 0.018, average: 176.5
episode: 555/30000, score: 343, e: 0.018, average: 178.2
episode: 556/30000, score: 97, e: 0.018, average: 174.9
episode: 557/30000, score: 107, e: 0.018, average: 174.8
episode: 558/30000, score: 117, e: 0.018, average: 174.2
episode: 559/30000, score: 251, e: 0.017, average: 176.1
episode: 560/30000, score: 91, e: 0.017, average: 174.2
episode: 561/30000, score: 67, e: 0.017, average: 164.6
episode: 562/30000, score: 144, e: 0.0

...

episode: 8267/30000, score: 212, e: 0.01, average: 101.5
episode: 8268/30000, score: 66, e: 0.01, average: 100.2
episode: 8269/30000, score: 144, e: 0.01, average: 101.3
episode: 8270/30000, score: 101, e: 0.01, average: 101.9
episode: 8271/30000, score: 95, e: 0.01, average: 102.2
episode: 8272/30000, score: 150, e: 0.01, average: 102.8
episode: 8273/30000, score: 88, e: 0.01, average: 103.3
episode: 8274/30000, score: 150, e: 0.01, average: 104.0
episode: 8275/30000, score: 68, e: 0.01, average: 103.4
episode: 8276/30000, score: 107, e: 0.01, average: 103.6
episode: 8277/30000, score: 83, e: 0.01, average: 103.9
episode: 8278/30000, score: 67, e: 0.01, average: 103.3
episode: 8279/30000, score: 130, e: 0.01, average: 104.2
episode: 8280/30000, score: 99, e: 0.01, average: 103.9
episode: 8281/30000, score: 75, e: 0.01, average: 104.1
episode: 8282/30000, score: 83, e: 0.01, average: 103.2
episode: 8283/30000, score: 150, e: 0.01, average: 104.5
episode: 8284/30000, score: 91, e: 0.01,

episode: 8413/30000, score: 91, e: 0.01, average: 107.7
episode: 8414/30000, score: 99, e: 0.01, average: 107.2
episode: 8415/30000, score: 115, e: 0.01, average: 107.3
episode: 8416/30000, score: 100, e: 0.01, average: 108.0
episode: 8417/30000, score: 159, e: 0.01, average: 109.8
episode: 8418/30000, score: 166, e: 0.01, average: 111.8
episode: 8419/30000, score: 137, e: 0.01, average: 111.8
episode: 8420/30000, score: 83, e: 0.01, average: 110.2
episode: 8421/30000, score: 104, e: 0.01, average: 110.1
episode: 8422/30000, score: 78, e: 0.01, average: 109.7
episode: 8423/30000, score: 66, e: 0.01, average: 109.5
episode: 8424/30000, score: 130, e: 0.01, average: 110.3
episode: 8425/30000, score: 101, e: 0.01, average: 110.0
episode: 8426/30000, score: 86, e: 0.01, average: 109.9
episode: 8427/30000, score: 75, e: 0.01, average: 110.0
episode: 8428/30000, score: 77, e: 0.01, average: 109.4
episode: 8429/30000, score: 91, e: 0.01, average: 107.8
episode: 8430/30000, score: 137, e: 0.01

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000001D2AB251F78> (for post_execute):


KeyboardInterrupt: 